In [1]:
import os
from pathlib import Path
import pandas as pd 
import re 

In [2]:
PROJECT_ROOT = Path.cwd().parent
os.chdir(PROJECT_ROOT)

In [3]:
X_train = pd.read_csv("data/processed/X_train.csv")
X_test  = pd.read_csv("data/processed/X_test.csv")
y_train = pd.read_csv("data/processed/y_train.csv").squeeze()
y_test  = pd.read_csv("data/processed/y_test.csv").squeeze()

In [6]:
def extract_title(name:str) ->str:
    if pd.isna(name):
        return 'Unknown'
    m = re.search(r",\s*([^\.]+)\.", name)
    return m.group(1).strip() if m else 'Unknown'
def add_features(df:pd.DataFrame) ->pd.DataFrame:
    df =df.copy()
    if 'SibSp' in df.columns and 'Parch' in df.columns:
        df ['FamilySize'] = df['SibSp'] + df['Parch'] +1
        df ['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    if 'Name' in df.columns:
        df['Title'] = df['Name'].apply(extract_title)
        rare = {"Lady","Countess","Capt","Col","Don","Dr","Major","Rev","Sir","Jonkheer","Dona"}
        df['Title'] = df['Title'].replace(list(rare),'Rare')
        df['Title'] = df['Title'].replace({"Mlle":"Miss", "Ms":"Miss", "Mme":"Mrs"})
    return df 


In [9]:
X_train_fe = add_features(X_train)
X_test_fe = add_features(X_test)

In [10]:
for col in ['Age','Fare']:
    if col in X_train_fe.columns:
        med = X_train_fe[col].median()
        X_train_fe[col] = X_train_fe[col].fillna(med)
        X_test_fe[col]  = X_test_fe[col].fillna(med)
for col in ["Sex", "Embarked", "Title"]:
    if col in X_train_fe.columns:
        X_train_fe[col] = X_train_fe[col].fillna("Missing")
        X_test_fe[col]  = X_test_fe[col].fillna("Missing")

In [11]:
drop_cols = [c for c in ['Name','Ticket'] if c in X_train_fe.columns]
X_train_fe = X_train_fe.drop(columns=drop_cols)
X_test_fe  = X_test_fe.drop(columns=drop_cols)

In [12]:
X_train_fe.to_csv("data/fe/X_train_fe.csv", index=False)
X_test_fe.to_csv("data/fe/X_test_fe.csv", index=False)
pd.Series(y_train).to_csv("data/fe/y_train.csv", index=False)
pd.Series(y_test).to_csv("data/fe/y_test.csv", index=False)